In [53]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from scipy.stats import pearsonr, spearmanr
from sklearn import cross_validation
from sklearn import linear_model
__author__ = 'amanda'


# positive_list = ['happy', 'friendly', 'sociable', 'kind', 'caring', 'calm', 'trustworthy', 'responsible', 
#                 'confident', 'humble', 'emotStable', 'normal', 'intelligent', 'interesting', 'emotional',
#                 'memorable', 'typical', 'familiar', 'common']
# negative_list = ['unhappy', 'unfriendly', 'introverted', 'mean', 'cold', 'aggressive', 'untrustworthy', 'irresponsible',
#                 'uncertain', 'egotistic', 'emotUnstable', 'weird', 'unintelligent', 'boring', 'unemotional', 
#                  'forgettable', 'atypical', 'unfamiliar', 'uncommon']

# np.savez('../clean_data/pos_neg_list', positive_list = positive_list, negative_list=negative_list)

tmp_file = np.load('../clean_data/pos_neg_list.npz')
pos_list = tmp_file['positive_list']
neg_list = tmp_file['negative_list']

In [66]:
social_feats = pd.read_csv('../clean_data/reordered_social_feature.csv')
# pos_feature = social_feats[positive_list]
# neg_feature = social_feats[negative_list]
attract_y = social_feats['attractive'].values
feature_x = social_feats.drop(['attractive', 'unattractive'], axis=1)

In [65]:
def comp_coefficient(feature_x, attract_y):
    
    feat_names = list(feature_x.columns.values)
    feature_x = feature_x.values

    # run ridge regression
    itr_num = 50
    coef_list = np.zeros((len(feat_names), itr_num))
    test_corr_list = np.zeros((itr_num, 1))

    for cur_itr in range(itr_num):
        x_train, x_test, y_train, y_test = cross_validation.train_test_split(feature_x, attract_y, test_size=0.5)
        clf = linear_model.RidgeCV(alphas=np.logspace(-3, 1, num=20))
        clf.fit(x_train, y_train)

        coef_list[:, cur_itr] = clf.coef_
        y_test_pred = clf.predict(x_test)
        corr = spearmanr(y_test, y_test_pred)
        test_corr_list[cur_itr] = corr[0]

    coef_mean = coef_list.mean(axis=1)
    df_coef = pd.DataFrame(coef_mean, index=feat_names, columns=['coefficient'])
    return df_coef, test_corr_list.mean()

In [67]:
for delete_feature_itr in range(19):
    cur_pos = positive_list[delete_feature_itr]
    cur_neg = negative_list[delete_feature_itr]
    
    drop_pos = feature_x.drop([cur_pos], axis=1)
    drop_neg = feature_x.drop([cur_neg], axis=1)
    drop_both = feature_x.drop([cur_pos, cur_neg], axis=1)
    
    df_coef_withall, test_corr_withall = comp_coefficient(feature_x, attract_y)
    df_coef_wo_pos, test_corr_wo_pos = comp_coefficient(drop_pos, attract_y)
    df_coef_wo_neg, test_corr_wo_neg = comp_coefficient(drop_neg, attract_y)
    df_coef_wo_both, test_corr_wo_both = comp_coefficient(drop_both, attract_y)
    
    print 'cur social feature pair:', cur_pos, cur_neg
    print 'correlation with all feature', test_corr_withall
    print 'correlation without', cur_pos, test_corr_wo_pos
    print 'correlation without', cur_neg, test_corr_wo_neg
    print 'correlation without both', test_corr_wo_both

cur social feature pair: happy unhappy
correlation with all feature 0.819696360626
correlation without happy 0.819196988043
correlation without unhappy 0.818241879164
correlation without both 0.816892888057
cur social feature pair: friendly unfriendly
correlation with all feature 0.817667049068
correlation without friendly 0.817306948429
correlation without unfriendly 0.817965795191
correlation without both 0.818602869513
cur social feature pair: sociable introverted
correlation with all feature 0.817441049688
correlation without sociable 0.815919220233
correlation without introverted 0.817546574143
correlation without both 0.81606455704
cur social feature pair: kind mean
correlation with all feature 0.817125500633
correlation without kind 0.816895258376
correlation without mean 0.815547468956
correlation without both 0.817917657221
cur social feature pair: caring cold
correlation with all feature 0.819423637056
correlation without caring 0.816170915357
correlation without cold 0.81689